In [1]:
%matplotlib inline

import os, re, glob
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join as opj
from tqdm import tqdm_notebook
from ast import literal_eval as make_tuple

sns.set_context('paper')

In [2]:
gsearch = lambda *args: glob.glob(opj(*args))
str_extract = lambda pattern, s: re.search(pattern, s).group(0)

In [3]:
data_dir = '../data/publicLifeLogData'
all_files = gsearch(data_dir, 'lifeLog_bigserver2*', '2*y.txt')
all_files.sort()
print(*all_files[:10], sep='\n')

../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_01January_29_Tuesday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_01January_30_Wednesday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_01January_31_Thursday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_01_Friday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_02_Saturday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_03_Sunday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_04_Monday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_05_Tuesday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_06_Wednesday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_07_Thursday.txt


In [4]:
def date_extract(s):
    
    date_regex = '([0-9]{4})_([0-9]{2})[A-Za-z]+_([0-9]{2})'
    date_search = re.findall(date_regex, s)
    date_str = ''.join(date_search[0])
    date_int = int(date_str)
    
    return date_int

In [5]:
def find_version(file):
    file_date = date_extract(file)

    tmp_ver = ver.copy()
    tmp_ver['lag'] = file_date - tmp_ver['release_date']
    tmp_ver = tmp_ver[tmp_ver['lag'] >= 0]

    file_ver = tmp_ver.loc[tmp_ver['lag'].idxmin()].version
    
    return file_ver

In [8]:
ver_file = '../1_download/outputs/version_history.tsv'

# Load file
ver = pd.read_csv(ver_file, sep='\t')

In [9]:
all_dates = [date_extract(f) for f in all_files]

In [10]:
start = 20190726

# Check if date is within range
between_dates = lambda d: (d >= start)

file_df = pd.DataFrame({'file': all_files,
                        'date': all_dates})
file_df['include'] = file_df.date.apply(between_dates)
file_df = file_df[file_df.include == True]

file_df.head()

,file,date,include
178,../data/publicLifeLogData/lifeLog_bigserver2.o...,20190726,True
179,../data/publicLifeLogData/lifeLog_bigserver2.o...,20190727,True
180,../data/publicLifeLogData/lifeLog_bigserver2.o...,20190728,True
181,../data/publicLifeLogData/lifeLog_bigserver2.o...,20190729,True
182,../data/publicLifeLogData/lifeLog_bigserver2.o...,20190730,True


In [ ]:
from tqdm import tqdm
data_files = file_df['file'].values
data_files

In [ ]:
data_list = []
empty_files = []
for i in tqdm(range(len(data_files))):
    f = data_files[i]
    print(f)
# for f in data_files:
    try:
        tmp_server = str_extract('(?<=lifeLog_)[a-zA-Z0-9]+', f)
        tmp_ver = find_version(f)
        
        tmp_d = pd.read_csv(f, sep =' ', header=None)
        tmp_d.insert(0, 'server', tmp_server)
        tmp_d.insert(0, 'release', tmp_ver)
        data_list.append(tmp_d)

    except:
        empty_files.append(f)

In [ ]:
raw_data = pd.concat(data_list)
raw_data.columns = ['release', 'server', 'event', 'timestamp', 'playerID', 'uniqueID', 'param1', 'param2',
                 'param3', 'param4', 'param5']
raw_data.head()

In [ ]:
birth_data = raw_data[raw_data['event'] == 'B'].copy()
birth_data = birth_data[['release','playerID', 'timestamp', 'uniqueID', 'param2','param5']]
birth_data = birth_data.rename({'param2': 'birth_location','param5': 'chain'}, axis='columns')
birth_data.head()

In [ ]:
death_data = raw_data[raw_data['event'] == 'D'].copy()
death_data = death_data[['release','playerID', 'timestamp', 'uniqueID', 'param3']]
death_data = death_data.rename({'param3': 'death_location'}, axis='columns')
death_data.head()


In [ ]:
loc_data = birth_data.merge(death_data, on = ['release','playerID','uniqueID'])
loc_data['birth'] = loc_data['birth_location'].apply(make_tuple).apply(np.array)
loc_data['death'] = loc_data['death_location'].apply(make_tuple).apply(np.array)

# Split coordinates (for easier plotting)
loc_data[['birthX', 'birthY']] = pd.DataFrame(loc_data['birth'].tolist(),
                                              index=loc_data.index)   
loc_data[['deathX', 'deathY']] = pd.DataFrame(loc_data['death'].tolist(),
                                              index=loc_data.index)

lifeLog = loc_data[['release','playerID','timestamp_x','timestamp_y','uniqueID','birthX','birthY','deathX','deathY','chain']]


lifeLog.head()

In [ ]:
# turn str under chain into num
digit = lambda x: int(x.split("=")[1])
lifeLog['num_chain'] = lifeLog['chain'].apply(digit)
lifeLogNew = lifeLog.drop(['chain'], axis=1)
lifeLogNew = lifeLogNew.rename({'timestamp_x': 'timestamp_birth', 'timestamp_y': 'timestamp_death'}, axis='columns')
lifeLogNew.head()

In [ ]:
##plot migration across versions
def plot_migration_across_releases(version):
    fig = plt.figure(figsize=(30,30))
    sns.scatterplot(data = lifeLogNew[lifeLogNew['release'] == version], x = 'birthX', y = 'birthY', 
                    hue ='timestamp_birth', palette="Blues", alpha =0.8)

    sns.scatterplot(data = lifeLogNew[lifeLogNew['release'] == (version+1)], x = 'birthX', y = 'birthY', 
                    hue ='timestamp_birth', palette="Reds", alpha =0.8)
    plt.title('version' + str(version) + '-' + str(version+1))
    
#     out_file = 'plots/migration_updates_v%s.png' % version
#     plt.savefig(out_file, dpi=100)

In [ ]:
##this function plots the birth and death location of each player
def plot_individual_immigration(sub_version):
    loc_subset = lifeLog[lifeLog['release'] == sub_version] ##plot migration for each version
    
    n_data = len(loc_subset)
    
    if n_data > 1: ##seems like it's missing data from some of the versions
        if n_data > 5000:
            a = 0.25
        else:
            a = 1

        fig = plt.figure(figsize=(30,30))
        for i, row in loc_subset.iterrows():
            plt.arrow(row['birthX'], row['birthY'],
                      row['deathX']-row['birthX'],
                      row['deathY']-row['birthY'],
                      alpha=0.2)
        sns.scatterplot(data = loc_subset, x = 'birthX', y = 'birthY', hue ='num_chain', alpha=a) #color indicates chain
        sns.scatterplot(data = loc_subset, x = 'deathX', y = 'deathY', marker = "+", hue ='num_chain', alpha=a) #marker type indicates birth/death 
        plt.xlabel('x')
        plt.ylabel('y')
    #     plt.ylim(-30000, 30000)
    #     plt.xlim(-50000, 100000)

        plt.title(sub_version)

#         out_file = 'plots/migration2_v%s.png' % sub_version
#         plt.savefig(out_file)

In [ ]:
# for s in np.arange(252,302):
#     plot_individual_immigration(s)
#     plt.show()

In [ ]:
for s in np.arange(280,302):
    plot_migration_across_releases(s)
    plt.show()